In [14]:
import gensim
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
from scipy.spatial.distance import cosine
from nltk.tokenize import PunktSentenceTokenizer

In [2]:
def seperatePunct(incomingString):
    newstring = incomingString
    newstring = newstring.replace("!"," ! ")
    newstring = newstring.replace("@"," @ ")
    newstring = newstring.replace("#"," # ")
    newstring = newstring.replace("$"," $ ")
    newstring = newstring.replace("%"," % ")
    newstring = newstring.replace("^"," ^ ")
    newstring = newstring.replace("&"," & ")
    newstring = newstring.replace("*"," * ")
    newstring = newstring.replace("("," ( ")
    newstring = newstring.replace(")"," ) ")
    newstring = newstring.replace("+"," + ")
    newstring = newstring.replace("="," = ")
    newstring = newstring.replace("?"," ? ")
    newstring = newstring.replace("\'"," \' ")
    newstring = newstring.replace("\""," \" ")
    newstring = newstring.replace("{"," { ")
    newstring = newstring.replace("}"," } ")
    newstring = newstring.replace("["," [ ")
    newstring = newstring.replace("]"," ] ")
    newstring = newstring.replace("<"," < ")
    newstring = newstring.replace(">"," > ")
    newstring = newstring.replace("~"," ~ ")
    newstring = newstring.replace("`"," ` ")
    newstring = newstring.replace(":"," : ")
    newstring = newstring.replace(";"," ; ")
    newstring = newstring.replace("|"," | ")
    newstring = newstring.replace("\\"," \\ ")
    newstring = newstring.replace("/"," / ")
    return newstring

def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

def text_cleaner(wordList):
    '''
    INPUT: List of words to be tokenized
    OUTPUT: List of tokenized words
    '''

    tokenziedList = []

    for word in wordList:

        #remove these substrings from the word
        word = word.replace('[deleted]','')
        word = word.replace('&gt','')

        #if link, replace with linktag
        if 'http://' in word:
            tokenziedList.append('LINK_TAG')
            continue

        #if reference to subreddit, replace with reddittag
        if '/r/' in word:
            tokenziedList.append('SUBREDDIT_TAG')
            continue

        #if reference to reddit user, replace with usertag
        if '/u/' in word:
            tokenziedList.append('USER_TAG')
            continue

        #if number, replace with numtag
        #m8 is a word, 5'10" and 54-59, 56:48 are numbers
        if hasNumbers(word) and not any(char.isalpha() for char in word):
            tokenziedList.append('NUM_TAG')
            continue

        #seperate puncuations and add to tokenizedList
        newwords = seperatePunct(word).split(" ")
        tokenziedList.extend(newwords)

    return tokenziedList

def mytokenizer(comment):
    '''
    Input: takes in a reddit comment as a str or unicode and tokenizes it
    Output: a tokenized list
    '''
    tokenizer = PunktSentenceTokenizer()
    sentenceList = tokenizer.tokenize(comment)
    wordList = []
    for sentence in sentenceList:
        wordList.extend(sentence.split(" "))

    return text_cleaner(wordList)

In [3]:
path1 = '../../data/labeledRedditComments.p'
path2 = '../../data/twitter-hate-speech-classifier.csv'
path3 = '../../data/RedditMay2015Comments.sqlite'

In [4]:
df = pickle.load(open(path1, 'rb'))


In [39]:
    # List of not hateful subreddits
final_nothate_srs = ['politics', 'worldnews', 'history', 'blackladies', 'lgbt',
                     'TransSpace', 'women', 'TwoXChromosomes', 'DebateReligion',
                     'religion', 'islam', 'Judaism', 'BodyAcceptance', 'fatlogic'
                     'gaybros','AskMen','AskWomen']
    # List of hateful subreddits
final_hateful_srs = ['CoonTown', 'WhiteRights', 'Trans_fags', 'SlutJustice',
                     'TheRedPill', 'KotakuInAction', 'IslamUnveiled', 'GasTheKikes',
                     'AntiPOZi', 'fatpeoplehate', 'TalesofFatHate','hamplanethatred'
                     'shitniggerssay','neofag','altright']

In [40]:
final_hateful_srs

['CoonTown',
 'WhiteRights',
 'Trans_fags',
 'SlutJustice',
 'TheRedPill',
 'KotakuInAction',
 'IslamUnveiled',
 'GasTheKikes',
 'AntiPOZi',
 'fatpeoplehate',
 'TalesofFatHate',
 'hamplanethatredshitniggerssay',
 'neofag',
 'altright']

In [42]:
df['subreddit'].value_counts()

worldnews          490354
fatpeoplehate      311183
politics           244927
AskMen             138839
AskWomen           137889
KotakuInAction     128156
TwoXChromosomes    105130
TheRedPill          59145
CoonTown            51979
DebateReligion      41015
islam               25443
history             25242
Judaism              9103
lgbt                 8253
TalesofFatHate       5239
AntiPOZi             4740
blackladies          4396
religion             2623
Trans_fags           2362
WhiteRights          1352
GasTheKikes           919
BodyAcceptance        579
women                 529
TransSpace            472
SlutJustice           309
IslamUnveiled         110
Name: subreddit, dtype: int64

In [43]:
df['label'].value_counts()

NotHate    1234794
Hate        565494
Name: label, dtype: int64

In [5]:
dfhate = pd.read_csv(path2)

In [26]:
dfhate.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech:confidence,_created_at,orig__golden,orig__last_judgment_at,orig__trusted_judgments,orig__unit_id,orig__unit_state,_updated_at,orig_does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech_gold,does_this_tweet_contain_hate_speech_gold_reason,does_this_tweet_contain_hate_speechconfidence,tweet_id,tweet_text
0,853718217,True,golden,86,NaN,The tweet uses offensive language but not hate...,0.6013,NaN,True,NaN,0.0,615561535.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,1.666196e+09,Warning: penny boards will make you a faggot
1,853718218,True,golden,92,NaN,The tweet contains hate speech,0.7227,NaN,True,NaN,0.0,615561723.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.295121e+08,Fuck dykes
2,853718219,True,golden,86,NaN,The tweet contains hate speech,0.5229,NaN,True,NaN,0.0,615562039.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,3.956238e+08,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,853718220,True,golden,98,NaN,The tweet contains hate speech,0.5184,NaN,True,NaN,0.0,615562068.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.975147e+08,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,853718221,True,golden,88,NaN,The tweet uses offensive language but not hate...,0.5185,NaN,True,NaN,0.0,615562488.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,5.889236e+08,@Zhugstubble You heard me bitch but any way I'...


In [27]:
dfhate['does_this_tweet_contain_hate_speech_gold'].value_counts()

The tweet contains hate speech\nThe tweet uses offensive language but not hate speech    25
The tweet is not offensive                                                               16
The tweet uses offensive language but not hate speech                                    13
The tweet uses offensive language but not hate speech\nThe tweet is not offensive         9
The tweet contains hate speech                                                            4
Name: does_this_tweet_contain_hate_speech_gold, dtype: int64

In [29]:
dfhate['orig_does_this_tweet_contain_hate_speech'].value_counts()

The tweet uses offensive language but not hate speech    24
The tweet contains hate speech                           22
The tweet is not offensive                               21
Name: orig_does_this_tweet_contain_hate_speech, dtype: int64

In [30]:
def myfunc(x):
    if x in ['The tweet is not offensive']:
        return 0
    else:
        return 1
    

In [31]:
dfhate['label'] = dfhate['does_this_tweet_contain_hate_speech'].map(lambda x: myfunc(x))

In [32]:
dfhate['label'].value_counts()

0    7274
1    7235
Name: label, dtype: int64

In [37]:
pre = np.array([1,1,1,0,0,0])
tru = np.array([1,0,1,0,1,1])

print sum(pre+tru == 2)
print sum(pre+tru == 0)
print sum(pre-tru == 1)
print sum(pre-tru == -1)

2
1
1
2


In [33]:
dfhate.to_csv(path2)

In [25]:
dfhate['does_this_tweet_contain_hate_speech'].value_counts()

The tweet is not offensive                               7274
The tweet uses offensive language but not hate speech    4836
The tweet contains hate speech                           2399
Name: does_this_tweet_contain_hate_speech, dtype: int64

In [6]:
model = gensim.models.Doc2Vec.load('base_model_original_tokenizer.doc2vec')

In [7]:
docvecs = model.docvecs

In [71]:
docvecs[0]

array([-0.12589121, -0.72398579, -0.13750482,  0.01074985, -0.07752338,
        0.13886024, -0.28057936, -0.11339198, -0.02804809,  0.08569955,
       -0.82156771, -0.2160503 ,  0.43471047,  0.19683866, -0.42664996,
        0.04931739, -0.33742675, -0.73014867, -0.12762018, -0.106265  ,
        0.39234036, -0.39817062, -0.14888453, -0.69462979, -0.30694923,
       -0.07339818, -0.18055308, -0.64332151,  0.57735366,  1.02309513,
       -0.34679177, -0.11108843, -0.05537489,  0.10253237,  0.31550717,
        0.52577186,  0.99049211,  0.05952468, -0.47226873, -0.05540276,
       -0.26681474, -0.08775025, -0.92837352, -0.13519953, -0.70518291,
       -0.20343725, -0.11048806,  0.15157135, -0.44448465,  0.43652859,
       -0.14788841, -0.7766152 , -0.3568432 , -0.55862421, -0.61907762,
       -0.49959502, -0.00634548, -0.02462127, -0.35062504, -0.07442575,
       -0.06620318, -0.2105757 ,  0.03848599,  0.07465713, -0.01480301,
        0.0909562 , -0.48590082, -0.48677263, -0.01644286,  0.53

In [20]:
doctags = docvecs.doctags

In [22]:
doctags.items()

[('SlutJustice', Doctag(offset=3, word_count=27446, doc_count=309)),
 ('AskWomen', Doctag(offset=25, word_count=6480218, doc_count=137889)),
 ('KotakuInAction', Doctag(offset=5, word_count=6416371, doc_count=128156)),
 ('blackladies', Doctag(offset=14, word_count=217549, doc_count=4396)),
 ('TwoXChromosomes', Doctag(offset=18, word_count=5929809, doc_count=105130)),
 ('politics', Doctag(offset=11, word_count=12084911, doc_count=244927)),
 ('AntiPOZi', Doctag(offset=8, word_count=221689, doc_count=4740)),
 ('islam', Doctag(offset=21, word_count=1520313, doc_count=25443)),
 ('GasTheKikes', Doctag(offset=7, word_count=47158, doc_count=919)),
 ('BodyAcceptance', Doctag(offset=23, word_count=39993, doc_count=579)),
 ('DebateReligion', Doctag(offset=19, word_count=3667567, doc_count=41015)),
 ('religion', Doctag(offset=20, word_count=197943, doc_count=2623)),
 ('worldnews', Doctag(offset=12, word_count=19291877, doc_count=490354)),
 ('fatpeoplehate', Doctag(offset=9, word_count=8193219, doc_

In [23]:
doctags['GasTheKikes']

Doctag(offset=7, word_count=47158, doc_count=919)

In [33]:
len(doctags['GasTheKikes'])

3

In [34]:
doctags['GasTheKikes'][0]

7

In [25]:
docvecs.count

26

In [26]:
type(docvecs)

gensim.models.doc2vec.DocvecsArray

In [28]:
type(docvecs[0])

numpy.ndarray

In [31]:
len(docvecs[0])

300

In [29]:
len(docvecs)

26

In [30]:
docvecs.index_to_doctag(0)

'CoonTown'

In [38]:
docvecs.most_similar(14)

[('BodyAcceptance', 0.21258679032325745),
 ('WhiteRights', 0.21146908402442932),
 ('TransSpace', 0.18464726209640503),
 ('AskMen', 0.16595458984375),
 ('islam', 0.1570722907781601),
 ('Judaism', 0.15705253183841705),
 ('TheRedPill', 0.15415698289871216),
 ('politics', 0.14485692977905273),
 ('KotakuInAction', 0.14440138638019562),
 ('religion', 0.14105652272701263)]

In [40]:
for i in xrange(len(docvecs)):
    print docvecs.index_to_doctag(i)
    print docvecs.most_similar(i)[0]
    print ""

CoonTown
('IslamUnveiled', 0.2738789916038513)

WhiteRights
('GasTheKikes', 0.38529878854751587)

Trans_fags
('SlutJustice', 0.41643446683883667)

SlutJustice
('Trans_fags', 0.41643446683883667)

TheRedPill
('WhiteRights', 0.22930307686328888)

KotakuInAction
('SlutJustice', 0.19015194475650787)

IslamUnveiled
('religion', 0.32894909381866455)

GasTheKikes
('AntiPOZi', 0.39256757497787476)

AntiPOZi
('GasTheKikes', 0.39256757497787476)

fatpeoplehate
('BodyAcceptance', 0.22982779145240784)

TalesofFatHate
('SlutJustice', 0.3041166663169861)

politics
('TransSpace', 0.26541823148727417)

worldnews
('IslamUnveiled', 0.22024142742156982)

history
('IslamUnveiled', 0.25044021010398865)

blackladies
('BodyAcceptance', 0.21258679032325745)

lgbt
('TransSpace', 0.33727318048477173)

TransSpace
('Trans_fags', 0.3732454180717468)

women
('IslamUnveiled', 0.3218139111995697)

TwoXChromosomes
('AskWomen', 0.368299663066864)

DebateReligion
('politics', 0.2580014169216156)

religion
('IslamUnveile

In [44]:
vocab = model.vocab

In [48]:
len(vocab.keys())

215221

In [49]:
vocab.keys()[:10]

[u"cake'",
 u'Craziness',
 u'nunnery',
 u'EXPLAIN',
 u'transend',
 u'sowell',
 u'Panzergroup',
 u'fleeces',
 u'woods',
 u'clotted']

In [50]:
vocab['sowell']

In [51]:
word = vocab['sowell']

In [55]:
word.index

107432

In [59]:
comment = 'hello world'.split(" ")
myvect = model.infer_vector(comment)
print myvect

[-0.04111374 -0.02500165  0.00240249 -0.05194312  0.01736004  0.03923237
 -0.02321895 -0.00029489  0.01171858 -0.00763664 -0.00423318 -0.01935696
  0.02693051  0.03966122 -0.0059855   0.01473564 -0.03234283 -0.01823245
  0.00217189 -0.00654919 -0.0153838  -0.01858403 -0.01150477  0.02978476
  0.0007985  -0.00109669  0.02316413  0.00655458  0.00364505 -0.01039089
 -0.04070766 -0.00415223  0.00397358 -0.03976255  0.00087709  0.01718757
  0.01112527 -0.01105386 -0.01857648  0.03114931  0.0043707   0.01010265
 -0.00273892 -0.0237     -0.01758535  0.00644217 -0.00706726  0.00386565
 -0.00940999 -0.01003488  0.00647073 -0.00198868  0.01122584 -0.00729731
 -0.01108328 -0.00288173 -0.03437344 -0.01104882 -0.00978341  0.00688382
 -0.02149723  0.02091461  0.01071058 -0.02032779 -0.03376953 -0.01208208
 -0.00408209 -0.05274599  0.00043536  0.00033211  0.01783239 -0.02652001
  0.00933295 -0.00230759  0.01432832  0.00012344 -0.00400857  0.01431063
  0.01445851  0.02728147  0.01825923  0.01869242  0

In [62]:
comment = ':)'.split(" ")
myvect = model.infer_vector(comment)
print myvect

[  1.24285102e-03  -1.29657832e-03   5.14305313e-04   1.31310441e-03
   3.30964016e-04   8.09322170e-04  -5.70763252e-04  -4.74735134e-04
   2.04503740e-04  -1.64516823e-04   1.21135334e-03   1.21228769e-03
  -1.41432253e-03  -3.14957608e-04   1.15435303e-03  -2.24736941e-04
  -1.38484733e-03  -1.29070238e-03   3.52830946e-04   5.44279872e-04
   1.10735476e-03  -1.04843464e-03  -3.41511914e-04   1.30341633e-03
  -6.44849555e-04  -1.29601581e-03  -1.07744410e-04   1.10998552e-03
   6.85571518e-04  -3.35332734e-04  -4.42845077e-04  -1.35943049e-03
  -1.56357617e-03  -1.17807882e-03   1.13402412e-03  -1.34440212e-04
  -3.37475707e-04  -1.51634391e-03   1.17436284e-03   6.81731792e-04
   5.00002818e-04  -1.15380972e-03   1.15671172e-03  -1.11719721e-03
   9.72568931e-04  -3.75860633e-04   3.36922676e-04  -6.96208794e-04
   6.73536677e-04   2.27944853e-04   1.40047050e-04  -2.13312713e-04
  -5.23253053e-04   3.54013377e-04   1.42755732e-03   1.69788735e-04
   8.25587544e-04  -1.22071803e-03

In [8]:
def mostSimilarDoc(model,comment):
    '''
    Input: doc2vec model, comment is a str
    Output: the label of the doc most similar to the comment
    '''
    
    docvecs = model.docvecs
    
    wordTokens = mytokenizer(comment)
#     wordTokens = comment.split(" ")
    commentVec = model.infer_vector(wordTokens)
    
    
    mostSimVec = None
    bestSimVal = None
    
    for vec_ind in xrange(len(docvecs)):
        simVal = 1 - cosine(commentVec,docvecs[vec_ind])
        
        if simVal>bestSimVal:
            mostSimVec = vec_ind
            bestSimVal = simVal
    
    
    return docvecs.index_to_doctag(mostSimVec), bestSimVal
    
        
        

In [20]:
random.sample(xrange(10), 11)

ValueError: sample larger than population

In [23]:
numsamps = 1000
randrows = random.sample(xrange(len(df.index)), numsamps)
comments = df.ix[randrows,'body'].values
subreddits = df.ix[randrows,'subreddit'].values
count = 0
for row,comment in enumerate(comments):
    predictedSub, simVal = mostSimilarDoc(model,comment)
    if predictedSub == subreddits[row]:
        count+=1
        
print count/float(len(comments))


0.187


In [23]:
predictedSub, simVal = mostSimilarDoc(model,'')
print predictedSub
print simVal

TwoXChromosomes
0.119427390737


In [10]:
wordTokens = "hi"

commentVec1 = model.infer_vector(wordTokens)
commentVec2 = model.infer_vector(wordTokens)

np.array_equal(commentVec1, commentVec2)

    

False